# Notebook to explore the tables created by dune_source using the default config in this repo
Before running this notebook, run the pipeline to create the tables in the duckdb database.:
```
python duckdb_pipeline.py
```
or 
```
python manual_resources_example.py
```

After running the pipeline, a duckdb database called `dune_source.duckdb` will be populated with the following tables:
- dex_volume
- dex_volume_incremental
- y2k_price_data
- custom_sql (with incremental cursor)

Important: make sure to close the connection to the duckdb database after running the notebook, if you don't, the connection will be locked and you won't be able to run the pipeline again.

In [61]:
import duckdb
db = duckdb.connect("dune_source.duckdb")

In [62]:
# dex_volume
db.execute("select * from dune_queries.dex_volume order by _col1").fetchdf()

,project,_col1,usd_volume,_dlt_load_id,_dlt_id
0,equalizer,2025-01-28 00:00:00.000 UTC,2.675506e+05,1740635232.399862,jZRdLxJDnLKDSA
1,nomiswap,2025-01-28 00:00:00.000 UTC,1.339799e+05,1740635232.399862,a6u/fhzo0hCOgw
2,sparkdex,2025-01-28 00:00:00.000 UTC,1.354083e+06,1740635232.399862,6qMxgcgQe+Nc0Q
3,swapr,2025-01-28 00:00:00.000 UTC,6.961598e+00,1740635232.399862,SGkZTLYS1NBpPQ
4,dracula_finance,2025-01-28 00:00:00.000 UTC,1.280415e+02,1740635232.399862,AejUKp44qGMExQ
...,...,...,...,...,...
4161,thena,2025-02-26 00:00:00.000 UTC,9.973215e+07,1740635232.399862,fUOpBrePFOyHmg
4162,bridgers,2025-02-26 00:00:00.000 UTC,NaN,1740635232.399862,UuCfWZYmv8imsw
4163,kyberswap,2025-02-26 00:00:00.000 UTC,1.298057e+05,1740635232.399862,GcyFftaXwKIrDA
4164,spartadex,2025-02-26 00:00:00.000 UTC,7.723550e+02,1740635232.399862,GVarlkND2UqcHg


In [63]:
# notice that the date column goes back to 2025-01-01 (starting_replication_value)
db.execute("select * from dune_queries.dex_volume_incremental order by date").fetchdf() 

,project,date,usd_volume,_dlt_load_id,_dlt_id
0,quickswap,2025-01-01 00:00:00.000 UTC,2.839669e+07,1740634777.175019,h33QnoAF90magQ
1,mummy_finance,2025-01-01 00:00:00.000 UTC,1.835366e+03,1740634777.175019,12TtV2+ZpXQNkw
2,fraxswap,2025-01-01 00:00:00.000 UTC,5.833909e+05,1740634777.175019,cChxHRhGzXBHWg
3,unichainswap,2025-01-01 00:00:00.000 UTC,3.503630e+03,1740634777.175019,hKPaTlVx5KxWVw
4,fenix,2025-01-01 00:00:00.000 UTC,6.620006e+03,1740634777.175019,ZtIBFDK+bEoC8Q
...,...,...,...,...,...
7987,1inch-LOP,2025-02-27 00:00:00.000 UTC,1.504364e+06,1740634777.175019,7T6tNnKVr67h0A
7988,mute,2025-02-27 00:00:00.000 UTC,2.619112e+04,1740634777.175019,hq8s3UJbwYE/Aw
7989,baseswap,2025-02-27 00:00:00.000 UTC,2.467560e+06,1740634777.175019,yLF8H8Icnqj1IA
7990,quickswap,2025-02-27 00:00:00.000 UTC,5.765113e+06,1740634777.175019,J13m2naFbo5X7g


In [64]:
# y2k_price_data (pass in Y2K as symbol to query param)
db.execute("select * from dune_queries.y2k_price_data").fetchdf()

,timestamp,year,month,blockchain,contract_address,symbol,decimals,price,_dlt_load_id,_dlt_id
0,2022-09-16 00:00:00.000 UTC,2022,9,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,0.000125,1740635232.399862,NMeNbKSbLKgU1w
1,2022-11-02 00:00:00.000 UTC,2022,11,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,0.000030,1740635232.399862,8MNEFtc6KsuXfg
2,2022-11-11 00:00:00.000 UTC,2022,11,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,0.000030,1740635232.399862,McliWz07tbV6PQ
3,2023-01-12 00:00:00.000 UTC,2023,1,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.687946,1740635232.399862,G5zRW3AZpP3+8g
4,2023-01-20 00:00:00.000 UTC,2023,1,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.646961,1740635232.399862,k/Auwr8LBNlZQA
...,...,...,...,...,...,...,...,...,...,...
4459,2024-12-21 00:00:00.000 UTC,2024,12,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.924233,1740635232.399862,3Xr30yaEp5mRFA
4460,2024-12-27 00:00:00.000 UTC,2024,12,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.575739,1740635232.399862,DWImnO/SxnevEQ
4461,2025-01-06 00:00:00.000 UTC,2025,1,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.706050,1740635232.399862,OICzgpCHVIB1hA
4462,2025-01-27 00:00:00.000 UTC,2025,1,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.382923,1740635232.399862,E6I+nHtu95u7zg


In [65]:
# custom_sql
db.execute("select * from dune_queries.custom_sql order by timestamp").fetchdf()

,timestamp,blockchain,contract_address,symbol,price,_dlt_load_id,_dlt_id
0,2024-11-02 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.083926,1740634777.175019,x7pfcwOX5EVVeg
1,2024-11-03 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.079143,1740634777.175019,nFOjxS6Yr0pQfA
2,2024-11-04 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.078314,1740634777.175019,lwZhs6kzpRD1gQ
3,2024-11-05 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.079181,1740634777.175019,fhUuyesAXqDoDQ
4,2024-11-06 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.089753,1740634777.175019,hrEZc1XV3u8Pdw
...,...,...,...,...,...,...,...
111,2025-02-22 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.050081,1740634777.175019,gE5MoswFmlvYjQ
112,2025-02-23 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.048287,1740634777.175019,XNvTfar6MRjtZg
113,2025-02-24 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.044149,1740634777.175019,Ss6qemgFCABopw
114,2025-02-25 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.038989,1740634777.175019,AvFP2Oy6BJirJA


In [66]:
# close the connection
db.close()